Credits:
1. [Keras - Bidirectional LSTM baseline ( lb 0.051)](https://www.kaggle.com/CVxTz/keras-bidirectional-lstm-baseline-lb-0-051/code)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.models import Model
from keras.layers import Dense, Embedding, Input
from keras.layers import LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
train.head()

In [ ]:
train_X = train["comment_text"].fillna("XX").values
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
train_Y = train[list_classes].values
test_X = test["comment_text"].fillna("XX").values

In [ ]:
maxlen = 120
max_features = 2000
token = text.Tokenizer(num_words=max_features)
token.fit_on_texts(list(train_X))
train_X = token.texts_to_sequences(train_X)
test_X = token.texts_to_sequences(test_X) 
train_X = sequence.pad_sequences(train_X, maxlen=maxlen)
test_X = sequence.pad_sequences(test_X, maxlen=maxlen)

In [ ]:
def get_model():
    embed_size = 128
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size)(inp)
    x = Bidirectional(LSTM(50, return_sequences=True))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(0.1)(x)
    x = Dense(50, activation="relu")(x)
    x = Dropout(0.1)(x)
    x = Dense(6, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
model = get_model()
batch_size = 32
epochs = 3
file_path="weights_base.best.hdf5"
checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=20)

callbacks_list = [checkpoint, early] #early
model.fit(train_X, train_Y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)


In [ ]:
model.load_weights(file_path)
preds = model.predict(test_X)
sample_submission = pd.read_csv("../input/sample_submission.csv")
sample_submission[list_classes] = preds

sample_submission.to_csv("starter_lstm.csv", index=False)